In [ ]:
import pandas as pd

# 1. Cargar archivo
df = pd.read_csv("sell-in.txt", sep="\t")
df["periodo"] = pd.to_datetime(df["periodo"], format="%Y%m")

# 2. Agrupar
df_grouped = df.groupby(["product_id", "periodo"])["tn"].sum().reset_index()

# 3. Pivotear
df_pivot = df_grouped.pivot(index="product_id", columns="periodo", values="tn")
df_pivot = df_pivot.reindex(sorted(df_pivot.columns), axis=1)

# 4. Agregar promedios últimos N meses con datos reales
df_result = df_pivot.copy()

def promedio_ultimos(data, n):
    return data.apply(lambda row: row.dropna().tail(n).mean(), axis=1)

df_result["prom_ultimos_3m"] = promedio_ultimos(df_result, 3)
df_result["prom_ultimos_6m"] = promedio_ultimos(df_result, 6)
df_result["prom_ultimos_12m"] = promedio_ultimos(df_result, 12)

# 5. Exportar
df_result.reset_index().to_csv("sellin_por_producto_con_promedios.csv", index=False)
print("✅ Archivo generado: sellin_por_producto_con_promedios.csv")
